In [ ]:
# Landmark Detection using CNN (Kaggle Dataset)

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from PIL import Image

# Kaggle setup for downloading dataset
!pip install kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"your_kaggle_username","key":"your_kaggle_api_key"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Download the Landmark Recognition Dataset from Kaggle
!kaggle competitions download -c landmark-recognition-2021
!unzip landmark-recognition-2021.zip

# Set dataset path
dataset_path = 'train'

data = []
labels = []

# Load images and labels (adjust based on dataset structure)
for class_id in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_id)
    if not os.path.isdir(class_path):
        continue
    for img_name in os.listdir(class_path)[:100]:  # Limit to 100 images per class for simplicity
        img_path = os.path.join(class_path, img_name)
        try:
            img = Image.open(img_path).resize((128, 128))
            data.append(np.array(img))
            labels.append(class_id)
        except Exception as e:
            continue

data = np.array(data) / 255.0
labels = pd.factorize(labels)[0]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(set(labels)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Plot accuracy and loss graphs
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()

plt.show()

# Save the trained model
model.save('landmark_detection_model.h5')

print("Landmark Detection Project Completed!")
